Como utilizar o bikepath_maker:

Comece com duas tabelas contendo os polígonos relevantes para a análise,
normalmente, usamos zonas censitárias e buffers relativos a pontos de interesse diversos.

Execute cada célula e veja seu resultado, altere conforme o desejado.

Parte 1 - Imports:

In [1]:
import geopandas as gpd
import pandas as pd
import ipywidgets as ipwd
import folium
import json
from shapely.geometry import LineString
import modules.pather as path
import modules.sorter as sort
from shapely import Point
import ipywidgets as widgets
from IPython.display import display
from time import sleep
import pyproj
import requests

Parte 2 - Criação do mapa e das zonas censitárias (polígonos) [a mensagem de erro abaixo deve ser um problema derivado da maneira com que o GeoPandas chama o método CRS da pyproj]

In [2]:
fmap = folium.Map(location=[-23.5, -46.6], zoom_start=10)

shapefile_path = 'files/demographics/SIRGAS_SHP_densidade_demografica_2010.shp'

polygons1 = gpd.read_file(shapefile_path)

polygons1.crs = {'init': 'epsg:31983'}
polygons1.to_crs(epsg='4326', inplace=True)

folium.Choropleth(
    geo_data=polygons1,
    name='choropleth',
    data=polygons1,
    columns=['id', 'populacao'],
    key_on='feature.properties.id',
    fill_color='OrRd',
    fill_opacity=0.9,
    line_opacity=0.1,
    legend_name='Densidade Populacional (hab/ha)'
).add_to(fmap)

C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Parte 3.1: Determinação do raio de distância máximo dos pontos

In [3]:
radius = 1000

def handle_slider_change_with_delay(change):
    def compute_new_radius():
        global radius
        radius = change.new
        print("New radius value:", radius)
    widgets.interact_manual(compute_new_radius)
    sleep(2)

slider = widgets.IntSlider(value=1000, min=1000, max=5000, step=250, description='Raio')
slider.observe(handle_slider_change_with_delay, names='value')

display(slider)


IntSlider(value=1000, description='Raio', max=5000, min=1000, step=250)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

Parte 3.2: Visualização do raio

In [4]:
gmap = folium.Map(location=[-23.526073932488359, -46.668179848334979], zoom_start=15)

folium.Circle(
    location=[-23.526073932488359, -46.668179848334979],
    radius=radius,
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.3,
).add_to(gmap)

gmap

Parte 3.3: Carregamento e plotting dos pontos de interesse

In [5]:
POI_file_path = "interest_points.geojson"
polygons2 = gpd.read_file(POI_file_path)

circle_group = folium.FeatureGroup(name='Circles')
for idx, point in polygons2.iterrows():
    folium.Circle(
        location=[point['geometry'].x, point['geometry'].y],
        radius=radius,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.3,
    ).add_to(circle_group)
    
circle_group.add_to(fmap)
# folium.LayerControl().add_to(fmap)
display(polygons2)

,latitude,longitude,geometry
0,-46.668180,-23.526074,POINT (-23.52607 -46.66818)
1,-46.648145,-23.539418,POINT (-23.53942 -46.64814)
2,-46.554376,-23.528401,POINT (-23.52840 -46.55438)
3,-46.639510,-23.549664,POINT (-23.54966 -46.63951)
4,-46.762041,-23.654882,POINT (-23.65488 -46.76204)
...,...,...,...
216,-46.726029,-23.488520,POINT (-23.48852 -46.72603)
217,-46.343630,-23.525415,POINT (-23.52541 -46.34363)
218,-46.536939,-23.637812,POINT (-23.63781 -46.53694)
219,-46.723682,-23.640078,POINT (-23.64008 -46.72368)


Parte 4: Criação das zonas de buffer e sobreposição de polígonos (sistema de cordenadas consertado)

In [6]:
dist_point_gdf = polygons2['geometry'][0]
transformer1 = pyproj.Transformer.from_crs('epsg:4326', 'epsg:32634', always_xy=True)
transformer2 = pyproj.Transformer.from_crs('epsg:32634', 'epsg:4326', always_xy=True)
new_x, new_y = transformer1.transform(dist_point_gdf.x, dist_point_gdf.y)
newer_x = new_x+radius
point_base = Point(new_x,new_y)
point_distant = Point(newer_x,new_y)
pb_4326 = Point(transformer2.transform(new_x, new_y))
pd_4326 = Point(transformer2.transform(newer_x, new_y))
radius2 = pb_4326.distance(pd_4326)

In [7]:
polygons2_buffer = gpd.GeoDataFrame(geometry=[])
polygons2_buffer.crs = {'init': 'epsg:4326'}

for index, row in polygons2.iterrows():
    coords = Point(row['geometry'].y, row['geometry'].x)
    buffer_geometry = coords.buffer(radius2)
    polygons2_buffer = pd.concat([polygons2_buffer, gpd.GeoDataFrame(geometry=[buffer_geometry])], ignore_index=True)

display(polygons2_buffer)

C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting o

C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not s

C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not s

C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not s

C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
C:\Users\jojos\.conda\envs\geo_env\Lib\site-packages\geopandas\array.py:1486: UserWarning: CRS not s

,geometry
0,"POLYGON ((-46.63701 -23.52607, -46.63716 -23.5..."
1,"POLYGON ((-46.61698 -23.53942, -46.61713 -23.5..."
2,"POLYGON ((-46.52321 -23.52840, -46.52336 -23.5..."
3,"POLYGON ((-46.60834 -23.54966, -46.60849 -23.5..."
4,"POLYGON ((-46.73087 -23.65488, -46.73102 -23.6..."
...,...
216,"POLYGON ((-46.69486 -23.48852, -46.69501 -23.4..."
217,"POLYGON ((-46.31246 -23.52541, -46.31261 -23.5..."
218,"POLYGON ((-46.50577 -23.63781, -46.50592 -23.6..."
219,"POLYGON ((-46.69252 -23.64008, -46.69267 -23.6..."


Parte 5: Ponte com o sorter

Organização 1:1

In [15]:
key_factor = 'populacao'

result11_gdf = sort.sorter11(polygons1, polygons2_buffer, key_factor)

display(result11_gdf)

,id,setor_cens,populacao,area_hect,habit_hect,ano_densid,an_censo,geometry,polygon2_center,polygon1_center
0,25824,355030850000092,1537.0,2.621578,586.288078,2010,2010,"POLYGON ((-46.67889 -23.50964, -46.67866 -23.5...",POINT (-46.66817984833497 -23.526073932488348),POINT (-46.677742385378195 -23.510106439106014)
1,27441,355030809000021,2240.0,10.76303,208.119838,2010,2010,"POLYGON ((-46.62547 -23.52636, -46.62558 -23.5...",POINT (-46.64814497895449 -23.539418218703233),POINT (-46.626669928873994 -23.528448792800553)
2,30815,355030820000113,1816.0,3.974456,456.917849,2010,2010,"POLYGON ((-46.54699 -23.55277, -46.54690 -23.5...",POINT (-46.554375722821725 -23.528401219245115),POINT (-46.546038475704016 -23.553993499444893)
3,22438,355030853000071,2419.0,20.104031,120.324127,2010,2010,"POLYGON ((-46.61034 -23.54913, -46.61127 -23.5...",POINT (-46.639509615583826 -23.54966442391953),POINT (-46.611360340877496 -23.549147777615133)
4,15000,355030817000177,2755.0,12.255191,224.80269,2010,2010,"POLYGON ((-46.78844 -23.64406, -46.78818 -23.6...",POINT (-46.76204141259482 -23.654882074829324),POINT (-46.785741585319954 -23.644482654182497)
...,...,...,...,...,...,...,...,...,...,...
183,16859,355030895000116,2640.0,16.132769,163.64209,2010,2010,"POLYGON ((-46.75824 -23.47572, -46.75823 -23.4...",POINT (-46.72602903811917 -23.48852010587342),POINT (-46.75644786880887 -23.474897564903902)
184,14294,355030836000244,1071.0,5.839867,183.394594,2010,2010,"POLYGON ((-46.36838 -23.50309, -46.36832 -23.5...",POINT (-46.34363027675714 -23.52541474730484),POINT (-46.368132002481026 -23.50408698881659)
185,13694,355030872000120,1790.0,3.552268,503.903426,2010,2010,"POLYGON ((-46.53788 -23.61241, -46.53799 -23.6...",POINT (-46.53693888564555 -23.637812160454352),POINT (-46.54023290598255 -23.610563976210273)
186,22990,355030883000038,2866.0,4.636704,618.111447,2010,2010,"POLYGON ((-46.73036 -23.61421, -46.73035 -23.6...",POINT (-46.72368198000666 -23.640078404364235),POINT (-46.72824336431382 -23.61395313798631)


Organização 1:N (bug dos 100)

In [8]:
polygons1 = polygons1.sort_values(by='habit_hect', ascending=False)
polygons1 = polygons1.dropna()
polygons1 = polygons1.reset_index(drop=True)

result1_gdf, result2_gdf = sort.sorter1n(polygons1, polygons2_buffer)

result1n_gdf = gpd.GeoDataFrame(geometry=[])
result1n_gdf['polygon1_center'] = result1_gdf['geometry']
result1n_gdf['polygon2_center'] = result2_gdf['geometry']
display(result1n_gdf)

,geometry,polygon1_center,polygon2_center
0,None,POINT (-46.52413 -23.58726),POINT (-46.52250 -23.55850)
1,None,POINT (-46.52413 -23.58726),POINT (-46.54437 -23.58929)
2,None,POINT (-46.52413 -23.58726),POINT (-46.52710 -23.60092)
3,None,POINT (-46.52413 -23.58726),POINT (-46.51611 -23.60287)
4,None,POINT (-46.52413 -23.58726),POINT (-46.53769 -23.59541)
...,...,...,...
126241,None,POINT (-46.70853 -23.83002),POINT (-46.72717 -23.82824)
126242,None,POINT (-46.50766 -23.48484),POINT (-46.49369 -23.44744)
126243,None,POINT (-46.50766 -23.48484),POINT (-46.48241 -23.48514)
126244,None,POINT (-46.50766 -23.48484),POINT (-46.52010 -23.49810)


Parte 6: Pather (escolha entre result11_gdf e result1n_gdf para rodar)

(Opção alternativa: carregar caminhos pré-calculados)

In [24]:
def graphopper_request(url, row):
    p_latitude = row['polygon1_center'].y
    p_longitude = row['polygon1_center'].x
    c_latitude = row['polygon2_center'].y
    c_longitude = row['polygon2_center'].x
    response = requests.get(url.format(p_latitude, p_longitude, c_latitude, c_longitude))
    if response.status_code == 200:
        json_data = response.json()
        return json_data
    else:
        raise Exception(f"Error: {response.status_code}")

def graphhopper_pather(POI_gdf, json_filename):

    graph_hopper_key = '5ff51c95-019f-422b-aab5-777d90552c28'

    url = 'https://graphhopper.com/api/1/route?key=' + graph_hopper_key + \
    '&point={},{}&point={},{}&vehicle=bike&elevation=true&type=json&points_encoded=false'

    gdf = gpd.GeoDataFrame(geometry=[])

    for index, row in POI_gdf.iterrows():
        data_from_request = graphopper_request(url, row)
        gdf = gdf.append(data_from_request, ignore_index=True)

    gdf = convert_list_columns_to_str(gdf)

    save_json(gdf, json_filename)

Parte 6.1: Defina um limite para o sorter 1n, por padrão ele itera todo o geodataframe na ordem definida

In [ ]:
# wanted_gdf = result1n_gdf
# wanted_gdf = result11_gdf
# limit = 50
file_name = "arquivo_caminhos"
graphhopper_pather(wanted_gdf, file_name, limit)

In [ ]:
with open("arquivo_caminhos.json", 'r') as file:
    data = json.load(file)

In [28]:
i = 0
while i < len(data['features']):
    coordinates = data['features'][i]['properties']['paths'][0]['points']['coordinates']
    linestring_coordinates = LineString([(point[0], point[1]) for point in coordinates])
    temp_gdf = gpd.GeoDataFrame(geometry=[linestring_coordinates])
    coordinates_gdf = pd.concat([coordinates_gdf, temp_gdf], ignore_index=True)
    folium.GeoJson(data={'type': 'Feature',
                          'geometry': linestring_coordinates.__geo_interface__},
                   name='LineString').add_to(fmap)
    i+=1